<a href="https://colab.research.google.com/github/adilsonsc/Coursera_Capstone/blob/master/battle_of_neighborhood_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading libraries**

In [40]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

ERROR! Session/line number was not unique in database. History logging moved to new session 59
Libraries imported.


# **Getting the inicial location coodinates**

In [41]:
#
address = 'SQN 109, Brasilia'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brasilia home are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Brasilia home are -15.7604133, -47.8868663.


In [0]:
neighborhood_latitude=-15.7663427
neighborhood_longitude=-47.8848694

## **Foursquare API and login infomations**

In [43]:
CLIENT_ID = 'I0E42DTBZAL3CE1INEJFYKTHLC1VFPCYOUFWSURMSQX3XYRP' # your Foursquare ID
CLIENT_SECRET = 'KMGDLJHW4IPBGQEMQDOFXBURDSN0UO22L1DS3O4WDFQGRUZ2' # your Foursquare Secret
VERSION = '20200327'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I0E42DTBZAL3CE1INEJFYKTHLC1VFPCYOUFWSURMSQX3XYRP
CLIENT_SECRET:KMGDLJHW4IPBGQEMQDOFXBURDSN0UO22L1DS3O4WDFQGRUZ2


In [45]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I0E42DTBZAL3CE1INEJFYKTHLC1VFPCYOUFWSURMSQX3XYRP&client_secret=KMGDLJHW4IPBGQEMQDOFXBURDSN0UO22L1DS3O4WDFQGRUZ2&v=20200327&ll=-15.7663427,-47.8848694&radius=1000&limit=500'

In [0]:
results = requests.get(url).json()
#results

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [48]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Belgrado Burger,Burger Joint,-15.765269,-47.886073
1,Beirute,Bar,-15.765288,-47.886226
2,Sabor Glacê,Dessert Shop,-15.765050,-47.886070
3,Pedacinho Pizzas,Pizza Place,-15.764996,-47.885723
4,Ernesto Cafés Especiais,Café,-15.764533,-47.885339
5,Oba Hortifruti,Fruit & Vegetable Store,-15.768837,-47.887648
6,Feitiço Mineiro,Mineiro Restaurant,-15.768755,-47.887004
7,Santo Frango,Brazilian Restaurant,-15.769807,-47.888565
8,Dulce Patagonia,Café,-15.762403,-47.888881
9,Komboleria,Gluten-free Restaurant,-15.770808,-47.884292


## **Map of Brasilia with venues near the place of reference, SQN 109**

In [52]:
# create map of Brasilia  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg


# **Getting the coodinates of the new location**

In [59]:

address2 = 'SQNW 307, Brasilia'

geolocator = Nominatim()
location = geolocator.geocode(address2)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of SQNW 307 are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of SQNW 307 are -15.7526112, -47.9118879.


In [0]:
neighborhood_latitude2=-15.7526112
neighborhood_longitude2=-47.9118879

## **Foursquare API and login infomations**

In [61]:
CLIENT_ID = 'I0E42DTBZAL3CE1INEJFYKTHLC1VFPCYOUFWSURMSQX3XYRP' # your Foursquare ID
CLIENT_SECRET = 'KMGDLJHW4IPBGQEMQDOFXBURDSN0UO22L1DS3O4WDFQGRUZ2' # your Foursquare Secret
VERSION = '20200329'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I0E42DTBZAL3CE1INEJFYKTHLC1VFPCYOUFWSURMSQX3XYRP
CLIENT_SECRET:KMGDLJHW4IPBGQEMQDOFXBURDSN0UO22L1DS3O4WDFQGRUZ2


In [63]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude2, 
    neighborhood_longitude2, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I0E42DTBZAL3CE1INEJFYKTHLC1VFPCYOUFWSURMSQX3XYRP&client_secret=KMGDLJHW4IPBGQEMQDOFXBURDSN0UO22L1DS3O4WDFQGRUZ2&v=20200327&ll=-15.7663427,-47.8848694&radius=1000&limit=500'

In [0]:
results = requests.get(url2).json()
#results

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Food Trucks Noroeste,Food Truck,-15.751135,-47.910346
1,Buongiorno,Bakery,-15.744515,-47.914302
2,Praça 108 Noroeste,Plaza,-15.751197,-47.910191
3,Academia,Gym / Fitness Center,-15.751007,-47.910370
4,Norusca Arena,Soccer Field,-15.752972,-47.909631
5,Academia Via Prestige,Gym / Fitness Center,-15.753208,-47.909248
6,Fitness Real Evolution,Gym / Fitness Center,-15.747930,-47.910376
7,Academia Fitness,Gymnastics Gym,-15.744920,-47.912181


## **Map of Brasilia with venues near the target place, SQNW 307**

In [67]:
# create map of Brasilia  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg
